Generate face bbox & face probs label for UTKFace images using MTCNN

Latest result:
- total: 23708; mtcnn fail count: 40; failure rate: 0.0016%
- total: 23708; mtcnn fail count: 60

In [1]:
# set up path to facenet_pytorch_c
import sys
sys.path.insert(1, '/home/ubuntu/mtcnn')

# facenet_pytorch_c: avoid confusion with system default facenet_pytorch
from facenet_pytorch_c import MTCNN


from tqdm import tqdm
import numpy as np
import os

# other custom scripts
import utils


In [2]:
# data loading parameters
workers = 4
resize_shape = (48, 48)


In [3]:
# get data
x_train, age_train, fname = utils.get_images(
    r'/home/ubuntu/UTKFace', resize_shape=resize_shape, label_mode=True
    )

100%|██████████| 23708/23708 [02:02<00:00, 192.95it/s]

Ignored images: 


In [4]:
# setup mtcnn

device = "cuda:0"

mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True, keep_all=True,
    device=device
)

In [28]:
# pyplot, for debugging

# saving numpy data in files
# https://stackoverflow.com/questions/28439701/how-to-save-and-load-numpy-array-data-properly

import torch
import matplotlib.pyplot as plt
from torchvision import transforms

idx = 0
fail = 0
total = 0

for img in (x_train):
    total += 1
    img = transforms.ToTensor()(img).unsqueeze(0)
    img = img.to(device)
    bbox, landmarks, prob, _ = mtcnn.onet(img)
    """
    if bbox is None:
        # mtcnn failed to detect a face
        fail += 1
        plt.imshow(x_train[idx]) # display failed image
        plt.show()
        print(fname[idx])
        bbox = np.zeros(4, dtype="int8") # set to int8, for indicating
        prob = np.zeros(2, dtype="int8") # this image is not used
        landmarks = np.zeros(10, dtype="int8")
    """;
    
    bbox = bbox.cpu().detach().numpy()
    prob = prob.cpu().detach().numpy()
    landmarks = landmarks.cpu().detach().numpy()
    
    #prob = np.array([0, prob[0]])
    np.save(fname[idx]+".bbox.npy", bbox)
    np.save(fname[idx]+".prob.npy", prob)
    np.save(fname[idx]+".landmarks.npy", landmarks)
    idx += 1

print("total: {}; mtcnn fail count: {}".format(total, fail))


total: 23708; mtcnn fail count: 0


In [ ]:
if bbox_train[4][0].dtype == np.dtype("float32"):
    print("yes")
else:
    print("no")


In [ ]:
print(prob[0])